In [45]:
import pandas as pd
import numpy as np
import os
import PIL
from pathlib import Path
from PIL import Image
import IPython
from IPython.display import display
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Convolution2D, MaxPooling2D
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

        

In [6]:
def image_to_pixel(img,maxPixel=32):
    imScalled = img.resize((maxPixel,maxPixel))
    img_pixel = list(imScalled.getdata())
    return img_pixel

#reading all images
train_path = "C:/Users/sj36580/Downloads/IIT Jodhpur/Sem 2/DL/assisgnment1/data/training"
validate_path = "C:/Users/sj36580/Downloads/IIT Jodhpur/Sem 2/DL/assisgnment1/data/validation"
train_image_df=pd.DataFrame(columns=['category','file_path'])
for folder in os.listdir(train_path):
    for fileNames in os.listdir(train_path+'/'+folder):
        row={'category':folder,'file_path':train_path+'/'+folder+'/'+fileNames}
        train_image_df.loc[len(train_image_df.index)] = [folder,train_path+'/'+folder+'/'+fileNames]
        
        #IPython.display.Image(data_path+'/'+folder+'/'+fileNames)

valid_image_df=pd.DataFrame(columns=['category','file_path'])
for folder in os.listdir(validate_path):
    for fileNames in os.listdir(validate_path+'/'+folder):
        row={'category':folder,'file_path':validate_path+'/'+folder+'/'+fileNames}
        valid_image_df.loc[len(valid_image_df.index)] = [folder,validate_path+'/'+folder+'/'+fileNames]
        
        #IPython.display.Image(data_path+'/'+folder+'/'+fileNames)



In [31]:
valid_image_df.head(1)
img = Image.open('C:/Users/sj36580/Downloads/IIT Jodhpur/Sem 2/DL/assisgnment1/data/training/aeroplane/2007_004009.jpg')
print(img.size)
img
maxPixel = 128
imScalled = img.resize((maxPixel,maxPixel))
img_pixel = np.reshape(list(imScalled.getdata()),(maxPixel,maxPixel,3))
img_pixel

(500, 366)


array([[[172, 186, 210],
        [170, 186, 210],
        [168, 184, 208],
        ...,
        [210, 210, 211],
        [210, 210, 211],
        [209, 209, 211]],

       [[171, 186, 209],
        [170, 186, 208],
        [169, 185, 207],
        ...,
        [211, 210, 214],
        [210, 210, 213],
        [209, 209, 211]],

       [[170, 186, 209],
        [170, 188, 209],
        [170, 188, 208],
        ...,
        [211, 211, 213],
        [211, 211, 213],
        [210, 210, 211]],

       ...,

       [[202, 202, 201],
        [205, 204, 201],
        [205, 205, 201],
        ...,
        [178, 195, 211],
        [177, 194, 212],
        [177, 194, 213]],

       [[202, 202, 201],
        [203, 203, 200],
        [204, 205, 201],
        ...,
        [186, 198, 209],
        [180, 196, 209],
        [178, 196, 210]],

       [[200, 200, 202],
        [203, 203, 202],
        [204, 204, 202],
        ...,
        [195, 202, 208],
        [189, 200, 208],
        [185, 198, 207]]

In [36]:
maxPixel = 128
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
valid_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory('C:/Users/sj36580/Downloads/IIT Jodhpur/Sem 2/DL/assisgnment1/data/training/',  class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])
valid_generator = valid_datagen.flow_from_directory('C:/Users/sj36580/Downloads/IIT Jodhpur/Sem 2/DL/assisgnment1/data/validation/',  class_mode = 'binary', target_size = (maxPixel, maxPixel),classes=['aeroplane','others'])

base_model = InceptionV3(input_shape = (maxPixel, maxPixel, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

Found 83 images belonging to 2 classes.
Found 723 images belonging to 2 classes.


16384